**Decision Trees Report**
Dawid Stasiak 148112
Milosz Matuszewski 148185


In [118]:
%pip install pydot-ng
import math

import pandas as pd
import pydot
import graphviz


[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


**Obliczanie entropii**
Pierwszym krokiem było obliczenie entropii dla całego zbioru danych. W tym celu należało obliczyć prawdopodobieństwo wystąpienia każdej z klas. Następnie należało obliczyć entropię dla każdej z klas. Ostateczna entropia została obliczona jako suma iloczynów prawdopodobieństwa wystąpienia danej klasy i entropii dla tej klasy.

In [119]:
def calculate_entropy(data):
    classes = data["Survived"].unique()
    entropy = 0
    total_samples = len(data)

    for c in classes:
        p = len(data[data["Survived"] == c]) / total_samples
        entropy += -p * math.log2(p)

    return entropy

**Obliczanie entropii warunkowej**
Następnie została obliczona entropia warunkowa dla każdego z atrybutów. Entropia warunkowa została obliczona jako suma iloczynów prawdopodobieństwa wystąpienia danej wartości atrybutu i entropii dla tej wartości.

In [120]:
def calculate_conditional_entropy(data, attribute):
    conditional_entropy = 0

    for value in data[attribute].unique():
        subset = data[data[attribute] == value]
        p_value = len(subset) / len(data)
        conditional_entropy += p_value * calculate_entropy(subset)

    return conditional_entropy

**Obliczanie information gain**
Następnie został obliczony information gain dla każdego z atrybutów. Information gain został obliczony jako różnica entropii całego zbioru danych i entropii warunkowej dla danego atrybutu.

In [121]:
def calculate_information_gain(data, attribute):
    return calculate_entropy(data) - calculate_conditional_entropy(data, attribute)

**Obliczanie gain ratio**
Następnie został obliczony gain ratio dla każdego z atrybutów. Gain ratio został obliczony jako iloraz information gain i entropii warunkowej dla danego atrybutu.

In [122]:
def calculate_gain_ratio(data, attribute):
    info_gain = calculate_information_gain(data, attribute)
    split_info = -sum(
        (len(data[data[attribute] == value]) / len(data))
        * math.log2(len(data[data[attribute] == value]) / len(data))
        for value in data[attribute].unique()
    )

    return info_gain / split_info

**Tworzenie drzewa decyzyjnego**
Przy pomocy utworzonych wsześniej funkcji zostało stworzone drzewo decyzyjne dla podanego zbioru danych

In [123]:
def build_decision_tree(data, depth=0):
    if len(data["Survived"].unique()) == 1:
        return {}

    if depth >= 5:
        return {}

    best_attribute = max(data.columns[:-1], key=lambda a: calculate_gain_ratio(data, a))
    tree = {best_attribute: {}}

    for value in data[best_attribute].unique():
        subset = data[data[best_attribute] == value]
        subset = subset.drop(columns=[best_attribute])
        subtree = build_decision_tree(subset, depth + 1)
        subtree["survived"] = len(subset[subset["Survived"] == 1])
        subtree["not survived"] = len(subset[subset["Survived"] == 0])
        tree[best_attribute][str(value)] = subtree

    return tree

**Wizualizacja drzewa decyzyjnego**
Ostatnim krokiem bylo stworzenie funckji do wizualizacji drzewa decyzyjnego. W tym celu została wykorzystana biblioteka pydot.

In [124]:
graph = pydot.Dot(graph_type='graph')

def draw(tree, parent=None):
    if len(tree.keys()) == 2:
        return
    node_name = list(tree.keys())[0]
    if parent is not None:
        node = pydot.Node(name=parent.get_name() + node_name, label=node_name, shape="box")
    else:
        node = pydot.Node(name=node_name, label=node_name, shape="box")
    graph.add_node(node)
    if parent is not None:
        graph.add_edge(pydot.Edge(parent, node))
    for k, v in tree[node_name].items():
        k_node = pydot.Node(name=node.get_name() + k,
                            label=k + "\nSurvived: " + str(v["survived"]) + "\nNot survived: " + str(v["not survived"]),
                            shape="box")
        graph.add_node(k_node)
        graph.add_edge(pydot.Edge(node, k_node))
        draw(v, parent=k_node)

**Wczytanie, przygotowanie danych i wywołanie funkcji**

In [125]:
data = pd.read_csv("titanic-homework.csv")
data = data.drop(columns=["PassengerId", "Name"])

data["Age"] = pd.cut(
    data["Age"],
    bins=[0, 20, 40, 100],
    labels=["young", "middle", "old"],
    right=False,
)
decision_tree = build_decision_tree(data)


draw(decision_tree)
graph.write_png('graph.png')
